In [1]:
import random
from datetime import datetime, date, time
from selenium import webdriver
from time import sleep
from functools import partial
import pandas as pd
from glob import glob
from os import path
import re
from hashlib import md5
from threading import Thread, Lock
from bs4 import BeautifulSoup
import requests

In [2]:
def random_sleep(a = 0, b = 0):
    pass

In [3]:
def save_data():
    global data
    global data_lock
    data_lock.acquire()
    
    df = pd.DataFrame(data)
    df.to_csv(path.join('data', r'avito_data{}.csv').format(str(datetime.now()).replace(':', '.')))
    
    for key in data.keys():
        data[key] = []
        
    data_lock.release()
    
def add_data(new_data):
    global data
    global data_lock
    data_lock.acquire()
    for key, item in normalize(new_data).items():
        data[key].append(item)
    data_lock.release()

In [4]:
def pop_account():
    global accounts
    global accounts_lock
    accounts_lock.acquire()
    while len(accounts) == 0:
        sleep(1)
    acc = accounts.pop(0)
    accounts_lock.release()
    print('got account', acc)
    return acc

In [5]:
def get_proxy_gen(file='proxy.txt'):
    while True:
        with open(file, 'r') as f:
            for proxy in f:
                yield proxy.strip()
def get_proxy():
    global proxer
    global proxer_lock
    proxer_lock.acquire()
    proxy = next(proxer)
    proxer_lock.release()
    return proxy

In [6]:
def get_new_browser(browser = None):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(r'--proxy-server=http://' + get_proxy())
    if browser is not None:
        try:
            browser.close()
        except Exception as e:
#             print(e)
            pass
    return webdriver.Chrome(options=chrome_options)

In [7]:
def get_url(browser, url):
    browser.get(url)
    while browser.title in ('Доступ временно заблокирован', 'www.avito.ru'):
        browser = get_new_browser(browser)
        browser.get(url)
    return browser

In [8]:
def get_visited_products(path=None):
    from os import path as path_
    if path is None:
        path = path_.join('data', 'visited_products.txt')
    try:
        with open(path, 'r') as file:
            return set([product.strip() for product in file])
    except IOError as e:
        print('error in get_visited_products function: ', e)
        return set()
def get_visited_accounts(directory='data', formatted='csv'):    
    try:
        urls = set()
        for file_name in glob(path.join(directory, '*.' + formatted)):
            df = pd.read_csv(file_name)
            for long_url in df.url:
                url = re.match(r"https://www.avito.ru/user/(.*)/profile", long_url).group(0)
                urls.add(url)
        return urls
    except Exception as e:
        print('error in get_visited_accounts function', e)
        return set()
def visit_product(url, path=None):
    from os import path as path_
    path = path or path_.join('data', 'visited_products.txt')

    global visited_products
    global visited_products_lock
    
    try:
        visited_products.add(url)
        with open(path, 'a') as file:
            print(url, file=file)
    except IOError as e:
        print('error in visit_product function: ', e)

In [9]:
def get_acc_from_product(i, url, results):
    while True:
        try:
            proxies = {
                'https': 'http://' + get_proxy(),
            }
            req = requests.get(url, proxies=proxies)
            if not req.ok or req.text.find('Доступ временно заблокирован') > 0:
                continue
        except Exception as e:
#             print(e)
            continue
        break
    
#     while req.text.find('Доступ временно заблокирован') > 0:
#         PROXY = next(proxer)
#         proxies = {
#             'https': 'http://' + PROXY,
#         }
#         try:
#             req = requests.get(url, proxies=proxies)
#         except Exception as e:
#             print(e)
            
    start = req.text.find('/user/')
    finish = req.text.find('/profile')
    if start >= 0 and finish >= 0:
        print('got account:', r'https://www.avito.ru' + req.text[start:finish+len('/profile')])
        results[i] = r'https://www.avito.ru' + req.text[start:finish+len('/profile')]

In [10]:
def get_accounts_from_page(url):
    #данные о проверенных товарах
    global visited_products

    # Переходим на страничку, откуда будем вытаскивать аккаунты из товаров
    while True:
        try:
            proxies = {
                'https': 'http://' + get_proxy(),
            }
            req = requests.get(url, proxies=proxies)
            if not req.ok or req.text.find('Доступ временно заблокирован') > 0:
                continue
        except Exception as e:
#             print(e)
            continue
        break
    
    soup = BeautifulSoup(req.text, 'html.parser')
    elements = soup.findAll('a', class_='item-description-title-link')
    urls = list(['https://www.avito.ru' + i.get_attribute_list('href')[0] for i in elements])
    
    # Ссылки, которые потом вернём
    accounts_list = []

    results = [None] * len(urls)
    
    for i in range(len(urls)):
        product_url = urls[i]
        if not (product_url in visited_products):
            visit_product(product_url)
            t = Thread(target=get_acc_from_product, args=(i, product_url, results))
            t.start()
    sleep(10)
    for i in results:
        if i is not None:
            accounts_list.append(i)
    
    return accounts_list

In [11]:
def filter_links(urls, HASH_MOD = [0]):
    '''Функция чекает список новых урлов на акки, отсеивает посещённые, оставляет нужные по хешу и тд и тп'''
    global visited_accounts
    result = []
    for url in urls:
        if not (url in visited_accounts) and url.startswith('https://www.avito.ru/user/') and \
                (int(md5(url.encode('utf-8')).hexdigest(), base=16) % 6 in HASH_MOD):
            result.append(url)
    return result

def search(url, HASH_MOD = [2, 3, 4], start_page = 1):
    '''url - ссылка на страничку авито, где есть товары и прокрутка страниц, чтобы можно было добавить
    "?p=<номер страницы> в конец url
    max_accounts - максимальное количество аккаунтов, которое вытаскивает функция search '''
    global proxer
    global accounts
    
    # Номер страницы на avito
    page = start_page
    
    # ссылки на аккаунты, собранные со странички avito
    links_from_page = []
    
    while True:
        while len(links_from_page) == 0:
            while (len(accounts) > 20):
                sleep(5)
            new_links = get_accounts_from_page(url=(url + r'?p={}').format(page))
            links_from_page = filter_links(new_links, HASH_MOD)
            page += 1

        account = links_from_page.pop()
        accounts.append(account)

In [ ]:
def parse():
    global accounts
    browser = get_new_browser()
    
    while True:
        url = pop_account()
        browser = get_url(browser, url)

        name = browser.find_element_by_class_name('profile-public-name-edUBF')
        data = browser.find_elements_by_class_name('profile-public-summary-item-wFbyw')
        prod = browser.find_elements_by_class_name('styles-date-2vN1V')
        cost = browser.find_elements_by_class_name('styles-price-1RC3V')
        i = 0
        l = len(prod) - 1
        l2 = l
        l3 = l2
        while(l != len(prod)):
            l3 = l2
            l = len(prod)
            l2 = l
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            random_sleep()
            prod = browser.find_elements_by_class_name('styles-date-2vN1V')
            cost = browser.find_elements_by_class_name('styles-price-1RC3V')
            try:
                element = browser.find_element_by_class_name('styles-button-10MP4')
                element.click()
                l = l - 1
                random_sleep()
            except Exception as e:
#                 print(e)
                pass
            if(l3 == len(prod)):
                l = len(prod)
        person_info = {
            'url': url,
            'name': name.text,
            'data': list(map(lambda x: x.text, data)),
            'prod_data_active': list(map(lambda x: x.text, prod)),
            'prod_cost_active': list(map(lambda x: x.text, cost))
        }
        c = browser.find_element_by_partial_link_text('Завершённые')
        c.click()
        random_sleep()
        prodold = browser.find_elements_by_class_name('styles-date-2vN1V')
        costold = browser.find_elements_by_class_name('styles-price-1RC3V')
        l = len(prodold) - 1
        l2 = l
        l3 = l2
        if browser.page_source.find('Нет завершённых объявлений') < 0:
            while(l != len(prodold)):
                l3 = l2
                l = len(prodold)
                l2 = l
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                random_sleep()
                prodold = browser.find_elements_by_class_name('styles-date-2vN1V')
                costold = browser.find_elements_by_class_name('styles-price-1RC3V')
                try:
                    element = browser.find_element_by_class_name('styles-button-10MP4')
                    element.click()
                    l = l - 1
                    random_sleep()
                except:
                    pass
                if(l3 == len(prod)):
                    l = len(prodold)

        person_info['prod_data_finished'] = list(map(lambda x: x.text, prodold))
        person_info['prod_cost_finished'] = list(map(lambda x: x.text, costold))
        
        add_data(person_info)


def strtodays(s):
    months = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
                  'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря'] 
    
    if s.lower().find('сегодня') >= 0:
        t = date.today()
        return date(t.year, t.month, t.day).toordinal()
    elif s.lower().find('вчера') >= 0:
        t = date.today() - timedelta(days=1)
        return date(t.year, t.month, t.day).toordinal()
    
    m = re.match(r'.*(\d\d\d\d).*', s)
    year = date.today().year if m is None else int(m.group(1))
    month = 0
    for i in range(12):
        if s.find(months[i]) >= 0:
            month = i + 1
            break
    day = int(re.match(r'(\d*) .*', s).groups(1)[0])

    return date(year, month, day).toordinal()


def normalize(account):
    result = dict()
    result['name'] = account['name'].lower()
    result['person_status'] = account['data'][0].lower()
    result['url'] = account['url']
    
    start_date = account['data'][1]
    if start_date.startswith('На Авито с '):
        months = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
                  'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря'] 
        month = 0
        for i in range(12):
            if start_date.find(months[i]) >= 0:
                month = i + 1
                break
        result['start_date'] = str(date(int(re.match('.*(\d\d\d\d).*', start_date).group(1)), month, 1).toordinal())
    else:
        result['start_date'] = ''
        
    result['answer_time'] = account['data'][3]
    result['first_sold_time'] = '' if len(account['prod_data_finished']) == 0 else str(strtodays(account['prod_data_finished'][-1]))
    result['n_active'] = str(len(account['prod_data_active']))
    result['n_finished'] = str(len(account['prod_data_finished']))
    result['active_finished_ratio'] = str((len(account['prod_data_active']) + 1) / (len(account['prod_data_finished']) + 1))
    
    active_costs = []
    for cost in account['prod_cost_active']:
        active_costs.append(int(re.match('(\d*).*', cost.replace(' ', '')).group(1)))
        
    finished_costs = []
    for cost in account['prod_cost_finished']:
        finished_costs.append(int(re.match('(\d*).*', cost.replace(' ', '')).group(1)))
        
    result['min_active_cost'] = str(min(active_costs)) if len(active_costs) > 0 else ''
    result['max_active_cost'] = str(max(active_costs)) if len(active_costs) > 0 else ''
    result['mean_active_cost'] = str(sum(active_costs) / len(active_costs)) if len(active_costs) > 0 else ''
    
    result['min_finished_cost'] = str(min(finished_costs)) if len(finished_costs) > 0 else ''
    result['max_finished_cost'] = str(max(finished_costs)) if len(finished_costs) > 0 else ''
    result['mean_finished_cost'] = str(sum(finished_costs) / len(finished_costs)) if len(finished_costs) > 0 else ''
    return result

url = r'https://www.avito.ru/rossiya/lichnye_veschi'
num_of_parsers = 2


accounts = []
proxer = get_proxy_gen()
data = {'name':[], 'min_active_cost':[], 'max_active_cost':[], 'mean_active_cost':[], 'min_finished_cost':[],
        'max_finished_cost':[], 'mean_finished_cost':[], 'answer_time':[], 'first_sold_time':[], 'n_active':[],
        'n_finished':[], 'active_finished_ratio':[], 'person_status':[], 'url':[], 'start_date':[]}

accounts_lock = Lock()
proxer_lock = Lock()
data_lock = Lock()

visited_products = get_visited_products()
visited_accounts = get_visited_accounts()

visited_products_lock = Lock()
visited_accounts_lock = Lock()

df = pd.DataFrame(data)

while True:
    try:
        searcher = Thread(target=search, args=(url, ))
        searcher.start()
        
        for _ in range(num_of_parsers):
            parser = Thread(target=parse)
            parser.start()
        
        while True:
            sleep(300)
            save_data()
            
    except:
        pass


got account: https://www.avito.ru/user/69f24f4ba37c07b2d490f6e86006c519/profile
got account: https://www.avito.ru/user/742892dfcf1514dd2ed2b5e3788673e6/profile
got account: https://www.avito.ru/user/6e45047155509c1d98523f740446d15a/profile
got account: https://www.avito.ru/user/6d7ce37987e114891bea86e91daaf118/profile
got account: https://www.avito.ru/user/33c8189ddc72b1d157c745946ef11d76/profile
got account: https://www.avito.ru/user/19151e8f461150be6011c19451a687f9/profile
got account: https://www.avito.ru/user/64917bc8736997527d9de0aac6ad186b/profile
got account: https://www.avito.ru/user/048913a814209ec775735e97fc4ae045/profile
got account: https://www.avito.ru/user/07211e4cc7a6a007acb646759425a126/profile
got account: https://www.avito.ru/user/8156e420313cc9cfde5a615af21e5386/profile
got account: https://www.avito.ru/user/cb9ebefcb298e286c0ed46bd84b3bfe1/profile
got account: https://www.avito.ru/user/35a8e497af98e73acc040077d4efcd67/profile
got account: https://www.avito.ru/user/c